In [1]:
import rospy
from human_moveit_config.human_model import HumanModel
from sensor_msgs.msg import JointState
from os.path import join
import json
from rospkg import RosPack
from baxter_commander.persistence import dicttostate
import re

import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

import socket
import binascii
import struct

from tools.udp_link import UDPLink 

In [2]:
rospy.init_node('testFeedback')

In [3]:
rospack = RosPack()
postures_file = join(rospack.get_path('reba_optim'), 'tmp', 'postures.json')
with open(postures_file) as datafile:
    postures_data = json.load(datafile)

In [4]:
pub = rospy.Publisher('reba_assess', JointState, queue_size=10)

In [5]:
human = HumanModel(control=True)

In [6]:
rate = rospy.Rate(5)

In [ ]:
####### SINGLE STATE TEST #######
nb_states = 1
state = human.get_random_state()
human.send_state(state)
for i in range(nb_states):
    pub.publish(state)
    rate.sleep()

In [ ]:
####### RANDOM STATE TEST #######
for i in range(10000):
    state = human.get_random_state()
    human.send_state(state)
    pub.publish(state)
    rate.sleep()

In [ ]:
####### RESET MODEL ########
T_state = human.get_initial_state()
human.send_state(T_state)

In [ ]:
####### CLASS STATE TEST #######
class_value = 1
states = postures_data[class_value - 1]
for s in states:
    state = dicttostate(s).joint_state
    human.send_state(state)
    pub.publish(state)   
    udp_send_state(state)

    rate.sleep()

In [7]:
udp = UDPLink()

In [8]:
def send_camera(cam_id):
    udp.send_int('/camera', cam_id)

In [10]:
def send_model(model_id):
    udp.send_int('/model', model_id)

In [25]:
def send_joint(joint, value, port=5005):
    vect = np.zeros(92)    
    vect[joint] = value
    udp.send_float_vector('/posture', vect)

In [12]:
def udp_send_state(joint_state, port=5005):
    # create a vector of 92 elements
    vect = np.zeros(92)
    
    
    # offset at T pose
    
    
    udp.send_float_vector(vect)

In [ ]:
human.send_state(state)
pub.publish(state)
udp_send_state(state)

In [ ]:
udp_send_state(T_state)

In [ ]:
state

In [23]:
send_camera(1)

In [24]:
send_model(1)

In [ ]:
send_joint()